ipyvuetify==1.3.0
ipywidgets==7.5.1
voila==0.1.21
torch==1.4.0
fastai==1.0.60
captum==0.2.0
pandas==1.0.3


# Test

In [1]:
import fastai.train
import pandas as pd
import torch
import torch.nn as nn
from captum.attr import LayerIntegratedGradients

# --- Model Setup ---

# Load a fast.ai `Learner` trained to predict IMDB review category `[negative, positive]`
awd = fastai.train.load_learner(".", "imdb_fastai_trained_lm_clf.pth")
awd.model[0].bptt = 200

# getting to the actual layer that holds embeddings
embedding_layer = awd.model[0]._modules["module"]._modules["encoder_dp"]

# working around the model prediction - first output only, apply softmax
forward_func = lambda x: torch.softmax(awd.model(x)[0], dim=-1)

# make integrated gradients instance
lig = LayerIntegratedGradients(forward_func, embedding_layer)

# Explainer logic


def get_attributions_for_sentence(
    sentence,
    awd_model=awd,
    lig_instance=lig,
    target=None,
    lig_n_steps=200,
    baseline_token="\n \n ",
):
    awd = awd_model
    lig = lig_instance
    vocab = awd.data.x.vocab
    sentence_tokens = awd.data.one_item(sentence)[0]
    reversed_tokens = [vocab.itos[w] for w in sentence_tokens[0]]
    baseline = (
        torch.ones_like(sentence_tokens) * vocab.stoi[baseline_token]
    )  # see "how to choose a good baseline"
    baseline[0, 0] = vocab.stoi["xxbos"]  # beginning of sentence is always #1
    y = awd.predict(sentence)
    if target is None:
        target = y[1].item()
    attrs = lig.attribute(sentence_tokens, baseline, target, n_steps=lig_n_steps)
    a = attrs.sum(-1)
    a = a / torch.norm(a)
    return (pd.Series(a.numpy()[0], index=reversed_tokens), y)

In [2]:
# https://www.imdb.com/review/rw5384922/?ref_=tt_urv
review_1917 = """I sat in a packed yet silent theater this morning and watched, what I believe to be, the next Academy Award winner for the Best Picture."""
"""I'm not at all a fan of war movies but I am a fan of great movies... and 1917 is a great movie. I have never been so mesmerized by set design and direction, the mass human emotion of this film is astonishingly captured and embedded magically in the audience. It keeps running through my mind...the poetry and beauty intertwined with the raw misery of war. Treat yourself... see this movie!
""";

In [3]:
import ipyvuetify as v
import ipywidgets as w

In [4]:
class Chip(v.Chip):
    positive = "0, 255, 0"
    negative = "255, 0, 0"

    def __init__(self, word, attribution):
        direction = self.positive if attribution >= 0 else self.negative
        color = f"rgba({direction}, {abs(attribution):.2f})"
        super().__init__(
            class_="mx-0 px-1",
            children=[word],
            color=color,
            value=attribution,
            label=True,
            small=True,
        )


def saliency_chips(attributions: pd.Series) -> v.ChipGroup:
    children = [Chip(w, a) for w, a in attributions.iteritems()]
    return v.ChipGroup(column=True, children=children)

In [5]:
@w.interact_manual(
    sentence=w.Textarea(review_1917),
    target=[None, 0, 1],
    baseline_token=["\n \n", ".", "<BOS>"],
)
def display_attributions(sentence="Great film", target=None, baseline_token="\n \n "):
    
    attributions, prediction = get_attributions_for_sentence(sentence)
    
    return saliency_chips(attributions)

interactive(children=(Textarea(value='I sat in a packed yet silent theater this morning and watched, what I be…